In [ ]:
from numba import cuda
device = cuda.get_current_device()
device.reset()

In [ ]:
import datetime
import glob
import os
import cv2
import time
import random
import shutil
import numpy as np
import matplotlib.pyplot as plt
from keras.models import model_from_json
from Model import CreateModel
%matplotlib inline
plt.ioff()

In [ ]:
train = CreateModel()
# Settings
net = 'CSRNet'
dataset = 'mixed_data'

test_img_paths = []
test_dm_paths = []

test_paths = 'Data/NewData/test/images'

for path in glob.glob(os.path.join(test_paths, '*.jpg')):
    test_img_paths.append(path)
    test_dm_paths.append(path.replace('.jpg', '.h5').replace('images', 'density-maps'))

test_x = train.gen_var_from_paths(test_img_paths[:], unit_len=None)
test_y = train.gen_var_from_paths(test_dm_paths[:], stride=8, unit_len=None)
test_x = train.norm_by_imagenet(test_x)
print('Test data size:', test_x.shape[0], test_y.shape[0], len(test_img_paths))



In [ ]:
import sys
import pandas as pd
from time import time, ctime

# Settings of display
dis_idx = 16 if dataset == 'mixed_data' else 0
dis_path = test_img_paths[dis_idx]
dis_x = test_x[dis_idx]
dis_y = test_y[dis_idx]
dis_lim = (5, 35) if dataset == 'mixed_data' else (40, 150)
time_st = time()

In [ ]:
dis_idx = 0
weights_dir = 'weights_mixed_data_EXP_0/CSRNet_best.hdf5'

model = model_from_json(open('models/{}.json'.format(net), 'r').read())
model.load_weights(weights_dir)
ct_preds = []
ct_gts = []


In [ ]:
for i in range(len(test_x[:])):
#     if i % 100 == 0:
    print('{}/{}'.format(i, len(test_x)))
    i += 0
    test_x_display = np.squeeze(test_x[i])
    test_y_display = np.squeeze(test_y[i])
    path_test_display = test_img_paths[i]
    try:
        pred = np.squeeze(model.predict(np.expand_dims(test_x_display, axis=0)))
    except Exception as e:
        print(test_img_paths[i])
        continue
    ct_pred = np.sum(pred)
    ct_gt = round(np.sum(test_y_display))
    ct_preds.append(ct_pred)
    ct_gts.append(ct_gt)
plt.plot(ct_preds, 'r>')
plt.plot(ct_gts, 'b+')
plt.legend(['ct_preds', 'ct_gts'])
plt.title('Pred vs GT')
plt.show()
error = np.array(ct_preds) - np.array(ct_gts)
plt.plot(error)
plt.title('Pred - GT, mean = {}, MAE={}'.format(
    str(round(np.mean(error), 3)),
    str(round(np.mean(np.abs(error)), 3))
))
plt.show()
idx_max_error = np.argsort(np.abs(error))[::-1]

In [ ]:
# Show the 5 worst samples
for worst_idx in idx_max_error[:5].tolist() + [dis_idx]:
    test_x_display = np.squeeze(test_x[worst_idx])
    test_y_display = np.squeeze(test_y[worst_idx])
    path_test_display = test_img_paths[worst_idx]
    pred = np.squeeze(model.predict(np.expand_dims(test_x_display, axis=0)))
    fg, (ax_x_ori, ax_y, ax_pred) = plt.subplots(1, 3, figsize=(20, 4))
    ax_x_ori.imshow(cv2.cvtColor(cv2.imread(path_test_display), cv2.COLOR_BGR2RGB))
    ax_x_ori.set_title('Original Image')
    ax_y.imshow(test_y_display, cmap=plt.cm.jet)
    ax_y.set_title('Ground_truth: ' + str(np.sum(test_y_display)))
    ax_pred.imshow(pred, cmap=plt.cm.jet)
    ax_pred.set_title('Prediction: ' + str(np.sum(pred)))
    plt.show()
    
# Show the 5 best samples
for worst_idx in idx_max_error[-5:].tolist() + [dis_idx]:
    test_x_display = np.squeeze(test_x[worst_idx])
    test_y_display = np.squeeze(test_y[worst_idx])
    path_test_display = test_img_paths[worst_idx]
    pred = np.squeeze(model.predict(np.expand_dims(test_x_display, axis=0)))
    fg, (ax_x_ori, ax_y, ax_pred) = plt.subplots(1, 3, figsize=(20, 4))
    ax_x_ori.imshow(cv2.cvtColor(cv2.imread(path_test_display), cv2.COLOR_BGR2RGB))
    ax_x_ori.set_title('Original Image')
    ax_y.imshow(test_y_display, cmap=plt.cm.jet)
    ax_y.set_title('Ground_truth: ' + str(np.sum(test_y_display)))
    ax_pred.imshow(pred, cmap=plt.cm.jet)
    ax_pred.set_title('Prediction: ' + str(np.sum(pred)))
    plt.show()

In [ ]:
lossMAE, lossRMSE, lossSFN, lossMAPE, PSNR, SSIM = train.eval_loss(
    model, test_x, test_y, quality=True
)
print(lossMAE, lossRMSE, lossSFN, lossMAPE, PSNR, SSIM)

In [ ]:
for idx in range(len(ct_preds)):
    test_x_display = np.squeeze(test_x[idx])
    test_y_display = np.squeeze(test_y[idx])
    path_test_display = test_img_paths[idx]
    pred = np.squeeze(model.predict(np.expand_dims(test_x_display, axis=0)))
    fg, (ax_x_ori, ax_y, ax_pred) = plt.subplots(1, 3, figsize=(20, 4))
    ax_x_ori.imshow(cv2.cvtColor(cv2.imread(path_test_display), cv2.COLOR_BGR2RGB))
    ax_x_ori.set_title('Original Image')
    ax_y.imshow(test_y_display, cmap=plt.cm.jet)
    ax_y.set_title('Ground_truth: ' + str(np.sum(test_y_display)))
    ax_pred.imshow(pred, cmap=plt.cm.jet)
    ax_pred.set_title('Prediction: ' + str(np.sum(pred)))
    plt.show()